In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('Games_Dataset_cleaned.csv')
df.head(3)

In [ ]:
df.drop(['Unnamed: 0.1','Unnamed: 0'], axis=1, inplace=True)
df.head(3)

In [ ]:
sns.set_palette("YlOrBr")

In [ ]:
genres = df['Genre'].str.split(', ').explode()
genres = genres.replace({'in Early Access': 'Early Access'}).value_counts()

genre_percentages = genres / genres.sum() * 100
others_count = genres[genre_percentages < 1].sum()

genres_filtered = genres[genre_percentages >= 1]
genres_filtered['Others'] = others_count

plt.figure(figsize=(10, 8))
genres_filtered.plot(kind='pie', autopct='%1.1f%%', startangle=140)
plt.title('Genre Distribution')
plt.ylabel('')  # Hide the y-label as it's not needed for pie charts
plt.tight_layout()
plt.show()

In [ ]:
def categorize_game(row):
    categories = row['Category']
    if 'Single-player' in categories and ('Multi-player' in categories or 'Co-op' in categories):
        return 'Both'
    elif 'Single-player' in categories:
        return 'Single-player'
    elif 'Multi-player' in categories or 'Co-op' in categories:
        return 'Multi-player'
    return 'Other'

In [ ]:
df['Game Type'] = df.apply(categorize_game, axis=1)

In [ ]:
sns.set_palette("pastel")

In [ ]:
df['Categories'] = df['Category'].str.split(',')
df_exploded = df.explode('Categories',ignore_index=True)
category_counts = df_exploded.groupby(['Game Type', 'Categories']).size()
top_5_categories = category_counts.groupby(level=0, group_keys=False).nlargest(5).reset_index(name='Counts')


In [ ]:
top_5_categories.pivot(index='Game Type', columns='Categories', values='Counts').plot(kind='bar', stacked=True)
plt.ylabel('Number of Games')
plt.title('Top Categories by Game Type')
plt.xticks(rotation=45)  # Rotate category names for better readability
plt.show()

In [ ]:
df['Release_Date'] = pd.to_datetime(df['Release_Date'], errors='coerce')
df_2021 = df[df['Release_Date'].dt.year == 2021]
df_2021['Release_Month'] = df_2021['Release_Date'].dt.month
monthly_releases = df_2021.groupby('Release_Month').size()

In [ ]:
monthly_releases.plot(kind='bar')
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

plt.xlabel('Month')
plt.ylabel('Number of Releases')
plt.title('Game Releases by Month')
plt.xticks(ticks=range(len(monthly_releases)), labels=months[:len(monthly_releases)], rotation=0)
plt.show()